In [ ]:
import sys
!{sys.executable} -m pip install sagemaker-experiments
!{sys.executable} -m pip install torch
!{sys.executable} -m pip install torchvision
!apt-get install -y make

import os
sess = boto3.Session()
account_id = sess.client('sts').get_caller_identity()["Account"]
os.environ['DATASETS_BUCKET'] = f'sagemaker-test-mnist-{sess.region_name}-{account_id}'
os.environ['DATASETS_KEY_PREFIX'] = 'mnist'
!make upload_datasets

In [ ]:
from sagemaker.pytorch import PyTorch
import sagemaker

estimator = PyTorch(
    source_dir='/root/sagemaker-pytorch-mnist',
    entry_point='main.py',
    output_path=f's3://{os.environ["DATASETS_BUCKET"]}/artifacts',
    role=sagemaker.get_execution_role(),
    framework_version='1.3.1',
    py_version='py3',
    train_instance_count=2,
    train_instance_type='ml.g4dn.xlarge',
    hyperparameters={
        'epochs': 10,
        'backend': 'gloo',
        'dropout': 0.2,
    },
    metric_definitions=[
        {'Name':'test:accuracy', 'Regex':'test accuracy: (.*?),'},
        {'Name':'test:loss', 'Regex':'test loss: (.*?);'}
    ],
    enable_sagemaker_metrics=True,
)

inputs = sagemaker.inputs.s3_input(f's3://{os.environ["DATASETS_BUCKET"]}/{os.environ["DATASETS_KEY_PREFIX"]}')

estimator.fit(inputs={'training': inputs})